In [ ]:
import os
import sys

import scipy as sp
from matplotlib import pyplot as plt
import numpy as np

CWD = os.path.abspath("")
sys.path.append(os.path.join(CWD, ".."))
from plt_settings import plt_settings

full_width = 5.5
ratio = 1 / 1.618

In [ ]:
ITERATIONS = int(5e5)

sampling_data = np.load(os.path.join(CWD, "1d_gaussian_vs_N.npz"))

pdlmc_samples = {}
pdlmc_samples[1] = sampling_data["pdlmc_x_1"]
pdlmc_samples[10] = sampling_data["pdlmc_x_10"]
pdlmc_samples[100] = sampling_data["pdlmc_x_100"]
pdlmc_samples[1000] = sampling_data["pdlmc_x_1000"]

pdlmc_lambda = {}
pdlmc_lambda[1] = sampling_data["pdlmc_lambda_1"]
pdlmc_lambda[10] = sampling_data["pdlmc_lambda_10"]
pdlmc_lambda[100] = sampling_data["pdlmc_lambda_100"]
pdlmc_lambda[1000] = sampling_data["pdlmc_lambda_1000"]

pdlmc_time = {}
pdlmc_time[1] = sampling_data["pdlmc_t_1"]
pdlmc_time[10] = sampling_data["pdlmc_t_10"]
pdlmc_time[100] = sampling_data["pdlmc_t_100"]
pdlmc_time[1000] = sampling_data["pdlmc_t_1000"]

In [ ]:
a, b = 1, 3

# True mean/variance
Z = sp.stats.norm.cdf(b) - sp.stats.norm.cdf(a)
true_mean = sp.integrate.quad(lambda x: x * sp.stats.norm.pdf(x) / Z, a, b)[0]
true_var = sp.integrate.quad(lambda x: x**2 * sp.stats.norm.pdf(x) / Z, a, b)[0] - true_mean**2

print(f"Mean of the distribution: {true_mean}/{true_var}")
print("")

# Estimated mean/variance
for N, samples in pdlmc_samples.items():
    print(
        f"[N = {N:>4}] Estimated mean/variance: {samples[:ITERATIONS // N].mean()}/{samples[:ITERATIONS // N].var()}",
        flush=True,
    )
print("")

# Out-of-support samples
for N, samples in pdlmc_samples.items():
    out_of_support = ((samples[: ITERATIONS // N] < a) | (samples[: ITERATIONS // N] > b)).mean()
    print(f"[N = {N:>4}] Percentage of out-of-support samples: {out_of_support * 100:.2f}%")

In [ ]:
plt_settings["figure.figsize"] = (full_width / 2, ratio * full_width / 2)

with plt.rc_context(plt_settings):
    _, axs = plt.subplots(1, 1, dpi=300)
    for N, samples in pdlmc_samples.items():
        cum_mean = np.cumsum(samples, axis=0) / np.arange(1, ITERATIONS + 1)
        axs.plot(
            cum_mean,
            label=rf"$N_b = {N}$ ({pdlmc_time[N]:.0f} s)",
        )
    axs.hlines(y=true_mean, xmin=0, xmax=ITERATIONS, linestyles="--", color="k", label="True mean")
    axs.ticklabel_format(scilimits=(0, 2))
    # axs.set_xlim(0, 1e5)
    axs.set_ylim(1.4, 1.6)
    axs.grid()
    axs.set_xlabel("PD-LMC iterations")
    axs.set_ylabel(r"Sample average")
    plt.legend(loc="lower right", ncol=2)

    plt.show()

In [ ]:
plt_settings["figure.figsize"] = (full_width / 2, ratio * full_width / 2)

with plt.rc_context(plt_settings):
    _, axs = plt.subplots(1, 1, dpi=300)
    for N, samples in pdlmc_samples.items():
        cum_mean = np.cumsum(samples, axis=0) / np.arange(1, ITERATIONS + 1)
        axs.plot(
            N * np.arange(ITERATIONS // N),
            cum_mean[: ITERATIONS // N],
            label=rf"$N_b = {N}$ ({pdlmc_time[N]:.0f} s)",
        )
    axs.hlines(y=true_mean, xmin=0, xmax=ITERATIONS, linestyles="--", color="k", label="True mean")
    axs.ticklabel_format(scilimits=(0, 2))
    axs.set_ylim(0.4, 1.6)
    axs.grid()
    axs.set_xlabel("LMC evaluations")
    axs.set_ylabel(r"Sample average")
    plt.legend(loc="lower right")

    plt.show()

In [ ]:
plt_settings["figure.figsize"] = (full_width / 2, ratio * full_width / 2)

with plt.rc_context(plt_settings):
    _, axs = plt.subplots(1, 1, dpi=300)
    for N, lmbdas in pdlmc_lambda.items():
        axs.plot(
            N * np.arange(ITERATIONS // N),
            lmbdas[: ITERATIONS // N],
            label=rf"$N_b = {N}$ ({pdlmc_time[N]:.0f} s)",
        )
    axs.ticklabel_format(scilimits=(0, 2))
    axs.grid()
    axs.set_xlabel("Iteration")
    axs.set_ylabel(r"Dual variable ($\lambda$)")
    plt.legend(loc="upper left")

    plt.show()